In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import sys
import scrapbook as sb

In [ ]:
#sys.path.append("/home/jupyter/packages/")
#import regenie_writer_functions.regenie_writer_functions as rw
from src import regenie_writer_functions as rw
from src import utils

In [ ]:
'''
Each run is identified by the run_name, cohort and phenotype
'''
run_name = "allofus_cad_eur"
cohort = "allofus_eur"
phenotype_names = "cad" #if multiple seperated by ';' should match exactly as given in associations_tracker sheet
metafile = "NA" #if reading from google sheet, metafile = "NA"

In [ ]:
from multiprocessing import cpu_count

# Get the number of CPU cores
num_cores = cpu_count() - 1
threads = min(31, num_cores) #after 31 cores performance starts to plateu, may be more useful to do xargs -P
print(threads)

In [ ]:
'''
GLOBAL VARS & READ IN CONFIG FILE
'''
path_2_config = "/home/jupyter/regenie_dependencies/config.json"
regenie_params = utils.get_config_parameter("regenie_defaults", path_2_config)
google_sheet = utils.get_config_parameter("google_sheet", path_2_config)
plink_path = utils.get_config_parameter("plink", path_2_config)["path"]

REGENIE = regenie_params["REGENIE_PATH"] #this should be present as an executable in your path
BSIZE = regenie_params["BSIZE"]
CASE_CONTROL_IMBALANCE = regenie_params["CASE_CONTROL_IMBALANCE_HANDLE"]

now = datetime.now()
dt_string = now.strftime("%Y_%m_%d__%H:%M")

base_output_file_name = "_".join([run_name, cohort, "_".join(phenotype_names.split(";"))])

In [ ]:
'''
Read in meta-file
'''
if metafile == "NA":
    google_sheet_id = google_sheet["id"]
    regenie_tracker_sheet = google_sheet["name"]
    regenie_tracker_sheet_url = google_sheet["link"].format(google_sheet_id, regenie_tracker_sheet)

    regenie_df = pd.read_csv(regenie_tracker_sheet_url)\
    .query("cohort == '{}'".format(cohort))\
    .query("phenotype_names == '{}'".format(phenotype_names))\
    .query("run_name == '{}'".format(run_name))\
    .reset_index()
else:
     regenie_df = pd.read_csv(metafile)\
     .query("cohort == '{}'".format(cohort))\
     .query("phenotype_names == '{}'".format(phenotype_names))\
     .query("run_name == '{}'".format(run_name))\
     .reset_index()

In [ ]:
#each run_name, cohort, phenotype combination should be unique in the tracking sheet
assert regenie_df.shape[0] == 1, "Values specified != 1"

In [ ]:
regenie_df

In [ ]:
'''
Read in all params
'''
input_params = regenie_df.to_dict()

'''
path params
'''
base_path = input_params["base_path"][0]
output_path = input_params["output"][0]

'''
phenotype params
'''
trait_type = input_params["trait_type"][0]
phenotype_files_string = input_params["phenotypes_file"][0]
samples_to_keep_file = input_params["samples_to_keep_file"][0]
rint = input_params["rint"][0]

'''
covariates params
'''
age_col_string = input_params["covariates:age"][0]
categorical_cols_string = input_params["covariates:categorical"][0]
binary_qt_cols_string = input_params["covariates:binary_qt"][0]
pc_covariates_string = input_params["covariates:PCs"][0]
quadratic_age_sex = input_params["add_quadratic_age_sex"][0]

'''
Step 1 specific params
'''
bgen_step1_file = input_params["bgen_step1"][0]
step1_vars = input_params["step1_vars"][0]
loco_predictions = input_params["loco_predictions"][0]
subset_step1_vars = input_params["subset_step1_file"][0]

'''
Step 2 specific params
'''
bgen_step2_file = input_params["bgen_step2"][0]
step2_vars = input_params["step2_vars"][0]
model = input_params["model"][0]
step2_run_type = input_params["step2_run_type"][0]

'''
Gene burden / Skat test params 
Currently only works for Gene burdens
'''
anno_file = input_params["anno_file"][0]
set_list = input_params["set_list"][0]
mask_defs = input_params["mask_defs"][0]
aafs = ",".join(str(input_params["aafs"][0]).split(";"))

In [ ]:
'''
Check required params exist
'''
required_params = [trait_type, base_path, output_path, phenotype_files_string]

required_params_check = [ x for x in required_params if pd.isna(x) ]

assert len(required_params_check) == 0, "required params missing"

In [ ]:
'''
Read in phenotype files
subset to cohort of interest
if samples file given, subset to samples of interest
'''
phenotype_files = phenotype_files_string.split(";")
phenotype_dfs = []
for phenotype_file in phenotype_files:
    temp_pheno = pd.read_csv(base_path + "/" + phenotype_file)
    if "cohort" in temp_pheno.columns: #subsetting to cohort to avoid multiple files with same name
        temp_pheno = temp_pheno.query("cohort == '{}'".format(cohort))
    phenotype_dfs.append(temp_pheno.set_index("sampleId"))

phenotype_df = pd.concat(phenotype_dfs, axis = 1, verify_integrity = True)\
.query("cohort == '{}'".format(cohort))

if pd.notnull(samples_to_keep_file):
    samples_df = pd.read_csv("/".join([base_path, samples_to_keep_file]), usecols = {"sampleId"}).set_index("sampleId")
    phenotype_df = pd.merge(phenotype_df, samples_df, on = ["sampleId"])

In [ ]:
'''
Parse covariates input and generate covariates file
IMPORTANT: Any row with null values will be dropped
'''

categorical_cols = rw.splitIfExists(categorical_cols_string)
age_col = rw.splitIfExists(age_col_string)[0]
binary_qt_cols = rw.splitIfExists(binary_qt_cols_string)
pc_covariates = rw.splitIfExists(pc_covariates_string)

pc_cols_to_add = []
for pc_col in pc_covariates:
    pc_covars = rw.processPCInput(pc_col)
    pc_cols_to_add.extend(pc_covars)

covariate_cols_merged = [age_col] + binary_qt_cols + categorical_cols + pc_cols_to_add

covariates_df = rw.createAnalysisReadyFiles(phenotype_df[covariate_cols_merged].dropna())

cat_covars = categorical_cols
covar_lists = [age_col] + binary_qt_cols + pc_covariates

'''
Add age^2 and age*sex columns if add_quadratic_age_sex is not specifically set to False
'''
if str(quadratic_age_sex).lower() != "no":
    covariates_df["age2"] = covariates_df[age_col] * covariates_df[age_col]
    covariates_df["age_sex"] = covariates_df[age_col] * covariates_df["sex"]
    covar_lists = covar_lists + ["age2", "age_sex"]
    
covariates_file_name = "/".join([base_path, output_path, base_output_file_name + "_covariates.tsv"])

covariates_df.to_csv(covariates_file_name, sep = "\t", index = False)
print("covariates DF: {}".format(covariates_file_name))
print("Categorial covariates: {}".format(",".join(cat_covars)))
print("Quant covariates: {}".format(",".join(covar_lists)))

In [ ]:
'''
Create phenotypes_df
'''
analysis_ready_phenotypes_df = rw.createAnalysisReadyFiles(phenotype_df[phenotype_names.split(";")].dropna())

phenotypes_file_name = "/".join([base_path, output_path, base_output_file_name + "_phenotypes.tsv"])
                                
analysis_ready_phenotypes_df.to_csv(phenotypes_file_name, sep = "\t", index = False)
print("Phenotypes DF: {}".format(phenotypes_file_name))

In [ ]:
analysis_ready_phenotypes_df

In [ ]:
'''
Add code below to view phenotypes and covariates
Get number of missing values overall cases/controls
If BT: median and means of cases and controls
If QT: pearson correlations and R-squared between outcome and covariates
'''

In [ ]:
'''
Write out step 1 command
'''
if str(loco_predictions) == 'nan':
    step1_command_file = "/".join([base_path, output_path, base_output_file_name + "_step1_command"])
    step1_f=open(step1_command_file, 'w+')
    #if needed subset variants to keep using plink
    variants_to_keep_step1_file = step1_vars
    
    if subset_step1_vars == 'yes':
        samples_file = "/".join([base_path, output_path, base_output_file_name + "_samples_in_analysis.tsv"])
        variants_to_keep_step1_file = "/".join([output_path, base_output_file_name + "_variants_to_keep.snplist"])
        
        analysis_ready_phenotypes_df["FID"].to_csv(samples_file, sep = "\t", index = False, header = False)
        print("## PLINK PRE-PROCESSING STEP ##", file=step1_f)
        print("{} \\".format(plink_path), file=step1_f)
        print("--bgen {}/{} ref-first \\".format(base_path, bgen_step1_file) , file=step1_f)
        print("--keep {} \\".format(samples_file) , file=step1_f)
        print("--extract {}/{} \\".format(base_path, step1_vars) , file=step1_f)
        print("--mac 50 --write-snplist \\", file=step1_f)
        print("--threads {} \\".format(threads) , file=step1_f)
        print("--out {}/{}".format(base_path, variants_to_keep_step1_file.split(".snplist")[0]), file=step1_f)
        print(" ", file=step1_f)
        print("## END PLINK PRE-PROCESSING STEP ##", file=step1_f)
        print(" ", file=step1_f)
    
    rw.printRegenieHouseKeeping(file_handle = step1_f, \
                             step = "1", \
                             phenotypes = phenotype_names, \
                             cohort = cohort, \
                             run_name = run_name, \
                             date_time = dt_string,
                             regenie_path = REGENIE,
                             threads = threads)

    rw.printRegenieExtractions(file_handle = step1_f, \
                            base_path = base_path, \
                            bgen_file = bgen_step1_file, \
                            vars_to_extract = variants_to_keep_step1_file, \
                            phenotypes_file_name = phenotypes_file_name, \
                            covariates_file_name = covariates_file_name, \
                            cat_covars = cat_covars, \
                            covar_lists = covar_lists, \
                            trait_type = trait_type.lower(), \
                            rint = str(rint).lower())
    '''
    print out step 1 specific commands
    '''
    print("--lowmem \\", file=step1_f)
    print("--loocv \\", file=step1_f)
    print("--step 1 \\", file=step1_f)
    print("--out {}/{}/{}".format(base_path, output_path, base_output_file_name), file=step1_f)
    
    step1_f.close()
    loco_predictions_file = "{}/{}/{}_pred.list".format(base_path, output_path, base_output_file_name)

else:
    loco_predictions_file = loco_predictions
    
print(loco_predictions_file)

In [ ]:
step2_command_file = "/".join([base_path, output_path, base_output_file_name + "_step2_command"])
step2_f=open(step2_command_file, 'w+')

if str(step2_run_type) == "nan" or str("step2_run_type") == "step2_only_w_loco":
    step2_results = "{}/{}/{}".format(base_path, output_path, base_output_file_name + "_sm_results")
    rw.printRegenieHouseKeeping(file_handle = step2_f, \
                             step = "2", \
                             phenotypes = phenotype_names, \
                             cohort = cohort, \
                             run_name = run_name, \
                             date_time = dt_string,
                             regenie_path = REGENIE,
                             threads = threads)

    rw.printRegenieExtractions(file_handle = step2_f, \
                            base_path = base_path, \
                            bgen_file = bgen_step2_file, \
                            vars_to_extract = step2_vars, \
                            phenotypes_file_name = phenotypes_file_name, \
                            covariates_file_name = covariates_file_name, \
                            cat_covars = cat_covars, \
                            covar_lists = covar_lists, \
                            trait_type = trait_type.lower(), \
                            rint = str(rint).lower())
    print("--minMAC 0.5 \\", file=step2_f) #put this in regeniestep2 function
    rw.printRegenieStep2(file_handle = step2_f, \
                         base_path = base_path, \
                         cohort = cohort, \
                         run_name = run_name, \
                         model = model, \
                         loco_predictions = loco_predictions_file, \
                         trait_type = trait_type.lower(), \
                         output_file = step2_results)
    step2_f.close()

In [ ]:
step2_gb_command_file = "/".join([base_path, output_path, base_output_file_name + "_step2_gb_command"])

if str(anno_file) != "nan":
    step2_gb_f=open(step2_gb_command_file, 'w+')
    step2_gb_results = "{}/{}/{}".format(base_path, output_path, base_output_file_name + "_gb_results")
    
    rw.printRegenieHouseKeeping(file_handle = step2_gb_f, \
                             step = "2 GeneBurdens", \
                             phenotypes = phenotype_names, \
                             cohort = cohort, \
                             run_name = run_name, \
                             date_time = dt_string,
                             regenie_path = REGENIE,
                             threads = threads)

    rw.printRegenieExtractions(file_handle = step2_gb_f, \
                            base_path = base_path, \
                            bgen_file = bgen_step2_file, \
                            vars_to_extract = step2_vars, \
                            phenotypes_file_name = phenotypes_file_name, \
                            covariates_file_name = covariates_file_name, \
                            cat_covars = cat_covars, \
                            covar_lists = covar_lists, \
                            trait_type = trait_type.lower(), \
                            rint = str(rint).lower())
    print("--minMAC 0.5 \\", file=step2_gb_f) #put this in regeniestep2 function
    rw.printRegenieStep2(file_handle = step2_gb_f, \
                         base_path = base_path, \
                         cohort = cohort, \
                         run_name = run_name, \
                         loco_predictions = loco_predictions_file, \
                         trait_type = trait_type.lower(), \
                         output_file = step2_gb_results, \
                         model = model, \
                         anno_file = anno_file, \
                         set_list = set_list, \
                         mask_defs = mask_defs, \
                         aafs = aafs)
    step2_gb_f.close()

In [ ]:
outputs = []
if str(loco_predictions) == 'nan':
    outputs.append("step 1 file written: {}".format(step1_command_file))
    print(step1_command_file)
if str(bgen_step2_file) != 'nan':
    outputs.append("step 2 file written: {}".format(step2_command_file))
    print(step2_command_file)
if str(anno_file) != "nan":
    outputs.append("step 2 gene_burden file written: {}".format(step2_gb_command_file))
    print(step2_gb_command_file)

In [ ]:
'''
Add code to post-GWAS QC
Add code to merge with annotations and add homRR|hetRA|homAA counts for cases and controls
'''

In [ ]:
sb.glue("command_files", outputs)